In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
from pathlib import Path
from openpyxl.styles import PatternFill
from openpyxl import load_workbook
from collections import defaultdict
import sys
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter
from numpy._core.numeric import indices

In [2]:
# cores list of sites for AA

project = 'Antarctic'
output_dir = '/Users/quinnmackay/Desktop/table_out'

# get all link combos
with open(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/parameters.yml') as f:
    data = yaml.safe_load(f)
list_sites = data["list_sites"]
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

error_margin = 0.1
big_error_margin = 0.25
minor_depth_inversion = 0.001
major_depth_inversion = 0.5
base_core_age = 'EDC'

In [3]:
# Create each segment
segment1 = np.arange(0.01, 0.1, 0.01)  # 0.01 to 0.09 by 0.01
segment2 = np.arange(0.1, 0.2, 0.02)   # 0.1 to 0.18 by 0.02
segment3 = np.arange(0.2, 0.51, 0.05)  # 0.2 to 0.5 by 0.05

# Concatenate them
bounds = np.concatenate([segment1, segment2, segment3])

total_errors = []

In [4]:
for error_margin in bounds:

    big_table = pd.DataFrame()
    all_links_count = {}
    all_links_foragesort = {}

    if base_core_age not in list_sites:
        print('Incorrect base_core set for sorting')
        sys.exit()

    for core in list_sites: # loop through each core
        for comparison_core in list_sites: # loop through each core other than the initial load
            pair = f"{core}-{comparison_core}"
            if core != comparison_core and pair in pairs: # make sure not the same core and we skip non-existent linkages
                pair_dir = Path(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/{pair}')

                # Check: directory exists AND contains at least one .txt file
                txt_files = list(pair_dir.glob("*.txt"))
                if not pair_dir.is_dir() or not txt_files:
                    continue

                dfs=[] #load all text files into one
                for txt in txt_files:
                    df = pd.read_csv(txt, sep="\t", comment="#")
                    dfs.append(df)
        
                num_files = len(dfs)
                load_data = pd.concat(dfs, ignore_index=True)
                original_rows = len(load_data)

                drop_rows = []
                drop_rows_merge = set()
                new_merged_rows = []
                for idx, row in load_data.iterrows():

                    mask1 = abs(row['depth1'] - load_data['depth1']) <= error_margin
                    mask1[idx] = False
                    mask2 = abs(row['depth2'] - load_data['depth2']) <= error_margin 
                    mask2[idx] = False

                    close_points = load_data[mask1 & mask2]
                    num_close = len(close_points)
                    close_idxs = load_data.index[mask1 & mask2]

                    if num_close > 0:
                        refs = [load_data.at[idx, 'reference']] + [load_data.at[i, 'reference'] for i in close_idxs] #adjoin references
                        merged_ref = "; ".join(str(r) for r in refs if pd.notna(r))

                        depth1_vals = [load_data.at[idx, 'depth1']] + [load_data.at[i, 'depth1'] for i in close_idxs]
                        merged_depth1 = np.mean(depth1_vals)

                        depth2_vals = [load_data.at[idx, 'depth2']] + [load_data.at[i, 'depth2'] for i in close_idxs]
                        merged_depth2 = np.mean(depth2_vals)

                        new_merged_rows.append({'reference': merged_ref, 'depth1': merged_depth1, 'depth2': merged_depth2}) #create new merged row

                        drop_rows_merge.add(idx)
                        for i in close_idxs:
                            drop_rows.append(i)
                            if drop_rows.count(i) >= num_files:
                                continue

                # drop duplicate rows
                drop_rows = set(drop_rows).union(drop_rows_merge)
                load_data = load_data.drop(index=drop_rows).reset_index(drop=True)
                # add merged rows
                merged_df = pd.DataFrame(new_merged_rows)
                load_data = pd.concat([load_data, merged_df], ignore_index=True)
                load_data.drop_duplicates(subset=['depth1', 'depth2'], inplace=True)
                load_data = load_data.reset_index(drop=True)

                load_data = load_data.sort_values(by=['depth1']).reset_index(drop=True)
            
                #set up pair code stuff
                load_data[f"{pair}_code"] = [f"{pair}_{idx}" for idx in load_data.index]

                if core or comparison_core == base_core_age: # get all links attached to base core for age sorting
                    if core == base_core_age:
                        all_links_count[comparison_core] = len(load_data)
                        all_links_foragesort[comparison_core] = load_data[['depth1', 'depth2']].copy() #get all links for sorting the age, all links attached to base core
                        all_links_foragesort[comparison_core].rename(columns={f'depth1': base_core_age, 'depth2': 'comparison_core'}, inplace=True)
                    elif comparison_core == base_core_age:
                        all_links_count[core] = len(load_data)
                        all_links_foragesort[core] = load_data[['depth1', 'depth2']].copy()
                        all_links_foragesort[core].rename(columns={f'depth2': base_core_age, 'depth1': 'comparison_core'}, inplace=True)

                # rename to create unique columns for this pair
                load_data = load_data.rename(columns={
                    'depth1': f"{pair}_{core}",
                    'depth2': f"{pair}_{comparison_core}",
                    'reference': f"{pair}_reference",
                })

                # append rows (block)
                big_table = pd.concat([big_table, load_data],
                                    axis=0,
                                    ignore_index=True)

    # make all_links_count the largest value
    all_links_count[base_core_age] = sum(all_links_count.values())

    core_groups = defaultdict(list)
    matching_groups = defaultdict(list)

    link_columns_temp = [col for col in big_table.columns if "reference" not in col] #remove ref cols
    link_columns = [col for col in link_columns_temp if "code" not in col] #remove code cols

    for col in link_columns:
        suffix = col.split("_")[-1]
        core_groups[suffix].append(col) #group cols by suffix
        
        match = col.split("_")[0]
        core1 = match.split("-")[0]
        core2 = match.split("-")[1]

        if core1 == suffix:
            matching_core = core2
        elif core2 == suffix:
            matching_core = core1

        matching_groups[suffix].append(f"{match}_{matching_core}")

    update_check = 0  # total number of filled-in values across all passes
    refresh = 1  # triggers loop until no new updates are found
    while refresh > 0:  # keep looping as long as new values were added
        refresh = 0  # reset per loop
        for core, assoc_cols in core_groups.items():  # columns associated with each core
            matching_cols = matching_groups[core]  # corresponding matching columns

            for col, match_col in zip(assoc_cols, matching_cols):  # pair actual vs matching column
                base_col = col.split("_")[0] # get base link name for reference column
                i_core1 = base_col.split("-")[0] #get core names for index updates
                i_core2 = base_col.split("-")[1]
                ref_col = f"{base_col}_reference"  # reference column name
                code_col = f"{base_col}_code"
                for col_check in assoc_cols:  # compare against all other columns of same core
                    if col == col_check:  # skip self-comparison
                        continue

                    col_updates = {}  # values to update in primary column
                    match_updates = {}  # values to update in matching column
                    ref_updates = {}  # values to update in reference column
                    code_updates = {}

                    for index, value in big_table[col].items():  # loop over each row
                        try:
                            diff = (big_table[col_check] - value).abs()  # compute absolute diff
                        except:
                            print(f'Error computing diff for {col} and {col_check} at index {index} with value {value}')
                            print(core_groups.items())
                            sys.exit()
                        matching_indices = diff[diff <= error_margin].index  # rows that agree within margin

                        for match_idx in matching_indices:  # for each compatible row
                            col_updates[match_idx] = big_table[col].at[index]  # schedule update for col
                            match_updates[match_idx] = big_table[match_col].at[index]  # schedule update for match_col
                            ref_updates[match_idx] = big_table[ref_col].at[index]  # schedule update for reference column
                            code_updates[match_idx] = big_table[code_col].at[index]
                
                    for match_idx, new_val in col_updates.items():  # apply col updates
                        if pd.isna(big_table.at[match_idx, col]):  # only fill empty cells
                            big_table.at[match_idx, col] = new_val  # write new value
                            update_check+=1  # count total updates
                            refresh+=1  # signal another full loop is needed
                    for match_idx, new_val in match_updates.items():  # apply match_col updates
                        if pd.isna(big_table.at[match_idx, match_col]):  # only fill empty cells
                            big_table.at[match_idx, match_col] = new_val  # write new value
                    for match_idx, new_val in ref_updates.items():  # apply reference column updates
                        if pd.isna(big_table.at[match_idx, ref_col]):  # only fill empty cells
                            big_table.at[match_idx, ref_col] = new_val  # write new value
                    for match_idx, new_val in code_updates.items():  # apply reference column updates
                        if pd.isna(big_table.at[match_idx, code_col]):  # only fill empty cells
                            big_table.at[match_idx, code_col] = new_val  # write new value
                
        print(f'total updates made: {update_check} (+{refresh})')  # show total and new updates this pass

    #deal with with duplicates
    non_ref_code_cols = [c for c in big_table.columns if "reference" not in c and "code" not in c]
    big_table[non_ref_code_cols] = big_table[non_ref_code_cols].round(8)
    duplicates_mask = big_table.duplicated(subset=non_ref_code_cols, keep='first')
    num_dupe = duplicates_mask.sum()
    big_table_cleaned = big_table.drop_duplicates(subset=non_ref_code_cols, keep='first').reset_index(drop=True)
    print(f'Reduced table by {num_dupe/len(big_table)*100:.2f}% due to duplicates')

    #reorganize based on ages
    core_chron = {}
    for core, columns in core_groups.items():
        file_path = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/Chronologies/{core}.txt'
        df = pd.read_csv(file_path, sep="\t", comment="#", names=['depth', 'age']).sort_values(by=['depth']).reset_index(drop=True)
        core_chron[core] = df

    chron_df = core_chron[base_core_age]
    for index, row in big_table_cleaned.iterrows():
        for core, columns in sorted(core_groups.items(),
                                key=lambda x: all_links_count[x[0]],
                                reverse=True): # For each core and its associated list of column names
            values = [] # Collect the values for this core on this row
            values = [row[col] for col in columns if not pd.isna(row[col])] # Remove NaN values so they don't interfere with comparison, get values for this core on this row
            if len(values) >= 1 and core == base_core_age:
                avg_depth = np.mean(values)
                age_core_row = np.interp(avg_depth, chron_df['depth'], chron_df['age'])
                big_table_cleaned.at[index, 'estimated_age'] = age_core_row
            elif len(values) >= 1 and pd.isna(big_table_cleaned.at[index, 'estimated_age']): #skip ones already done 
                avg_depth = np.mean(values)
                equiv_depth = np.interp(avg_depth, all_links_foragesort[core]['comparison_core'], all_links_foragesort[core][base_core_age])
                age_core_row = np.interp(equiv_depth, chron_df['depth'], chron_df['age'])
                big_table_cleaned.at[index, 'estimated_age'] = age_core_row

    big_table_cleaned = big_table_cleaned.sort_values(by=['estimated_age']).reset_index(drop=True)
    big_table_cleaned = big_table_cleaned.drop(columns=['estimated_age'])

    # Do evaluation for errors (inter-row errors)
    within_row_errors = []
    within_row_errors_core = []
    within_row_big_errors = []
    within_row_big_errors_core = []
    for index, row in big_table_cleaned.iterrows(): # Iterate over every row in the table
        for core, columns in core_groups.items(): # For each core and its associated list of column names
            values = []
            for col in columns: # Collect the values for this core on this row
                values.append(row[col])
            values = [v for v in values if not pd.isna(v)] # Remove NaN values so they don't interfere with comparison
            if len(values) >= 2:
                diff = abs(max(values) - min(values))
                if diff >= error_margin:
                    within_row_errors.append(index)
                    within_row_errors_core.append(core)
                    #print(f"Row {index} core {core} values: {values} diff={diff}")
                if diff >= big_error_margin and index not in within_row_big_errors:
                    within_row_big_errors.append(index)

    # Do evaluation for depth inversion errors 
    depth_inversion_errors = []
    depth_inversion_errors_core = []
    depth_inversion_errors_major = []
    for core, columns in core_groups.items():  
        for col in columns:  # Check each depth column for this core
            # Extract the column's non-NaN values with their index
            series = big_table_cleaned[col].dropna()
            # Compare each depth with the previous one in sorted index order
            inversion_detect = False
            last_clean = series.iloc[0]  # Initialize last clean depth value
            for (prev_idx, prev_val), (curr_idx, curr_val) in zip(series.items(), list(series.items())[1:]):
                
                if curr_val > prev_val and inversion_detect==False:
                    last_clean = curr_val

                if inversion_detect==True:
                    if curr_val >= prev_val and curr_val >= last_clean:
                        inversion_detect=False
                        last_clean = curr_val
                    elif curr_val >= prev_val and curr_val < last_clean-minor_depth_inversion: #capture still inverted values just above previous error
                        depth_inversion_errors.append(curr_idx)
                        depth_inversion_errors_core.append(core)
                
                if curr_val < prev_val-minor_depth_inversion:  # inversion detected
                    depth_inversion_errors.append(curr_idx)
                    depth_inversion_errors_core.append(core)
                    inversion_detect=True

                if curr_val < prev_val-major_depth_inversion and curr_idx not in depth_inversion_errors_major:
                    depth_inversion_errors_major.append(curr_idx)

    #move cols around
    reference_cols = [c for c in big_table_cleaned.columns if "reference" in c]
    code_cols = [c for c in big_table_cleaned.columns if "code" in c]
    other_cols = [c for c in big_table_cleaned.columns if "reference" not in c and "code" not in c]
    big_table_cleaned = big_table_cleaned[other_cols + reference_cols + code_cols]

    rename_map = {}
    for suffix, cols in core_groups.items():
        for col in cols:
            rename_map[col] = suffix  # rename to suffix only
    big_table_cleaned.rename(columns=rename_map, inplace=True)

    total_errors.append(len(within_row_errors))
    print('----------')
    print(f'Total within-row errors at error margin {error_margin}: {len(within_row_errors)}')
    print('----------')


Processed pair EDC-WDC, total points after merging: 714, (897 original total rows)
Called by row 791.88 | 1454.37 from reference AICC2012_Links.
Processed pair EDC-EDML, total points after merging: 555, (828 original total rows)
Processed pair EDC-DF, total points after merging: 1555, (1605 original total rows)
Processed pair EDC-TALDICE, total points after merging: 232, (246 original total rows)
Processed pair WDC-EDML, total points after merging: 1117, (1315 original total rows)
Processed pair WDC-DF, total points after merging: 845, (1078 original total rows)
Processed pair WDC-TALDICE, total points after merging: 797, (1219 original total rows)
Processed pair EDML-DF, total points after merging: 215, (215 original total rows)
Processed pair EDML-TALDICE, total points after merging: 128, (128 original total rows)
Processed pair DF-TALDICE, total points after merging: 111, (111 original total rows)
total updates made: 18269 (+18269)
total updates made: 20663 (+2394)


KeyboardInterrupt: 